# In this case study lets implement Ensemble Stacking & voting regressor on boston dataset

# About Dataset

The Boston Housing Dataset

The Boston Housing Dataset is a derived from information collected by the U.S. Census Service concerning housing in the area of Boston MA. The following describes the dataset columns:

* CRIM - per capita crime rate by town
* ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS - proportion of non-retail business acres per town.
* CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
* NOX - nitric oxides concentration (parts per 10 million)
* RM - average number of rooms per dwelling
* AGE - proportion of owner-occupied units built prior to 1940
* DIS - weighted distances to five Boston employment centres
* RAD - index of accessibility to radial highways
* TAX - full-value property-tax rate per 10,000 dollors
* PTRATIO - pupil-teacher ratio by town
* B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
* LSTAT - % lower status of the population
* MEDV - Median value of owner-occupied homes in 1000 dollor's

## Import Section

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sks
from sklearn.datasets import load_boston
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,median_absolute_error,mean_squared_error,mean_absolute_percentage_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector 
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

Lets load the data

In [2]:
boston = load_boston()
print(boston.data.shape)

(506, 13)


In [3]:
print(boston.feature_names)

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [4]:
bos = pd.DataFrame(boston.data)
print(bos.head())

        0     1     2    3      4      5     6       7    8      9     10  \
0  0.00632  18.0  2.31  0.0  0.538  6.575  65.2  4.0900  1.0  296.0  15.3   
1  0.02731   0.0  7.07  0.0  0.469  6.421  78.9  4.9671  2.0  242.0  17.8   
2  0.02729   0.0  7.07  0.0  0.469  7.185  61.1  4.9671  2.0  242.0  17.8   
3  0.03237   0.0  2.18  0.0  0.458  6.998  45.8  6.0622  3.0  222.0  18.7   
4  0.06905   0.0  2.18  0.0  0.458  7.147  54.2  6.0622  3.0  222.0  18.7   

       11    12  
0  396.90  4.98  
1  396.90  9.14  
2  392.83  4.03  
3  394.63  2.94  
4  396.90  5.33  


In [5]:
bos['PRICE'] = boston.target

X = bos.drop('PRICE', axis = 1)
y = bos['PRICE']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(404, 13)
(102, 13)
(404,)
(102,)


In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
result_dataset = pd.DataFrame(data =np.zeros((6,12)) , index = ['r2_score','adjusted_r2_score','median_absolute_error','mean_squared_error','mean_absolute_percentage_error','mean_absolute_error'],columns = ['Knearest_neighbours','linear_regression','polynomial_linear_regression','lasso_regression','ridge_regression','elasticnet_regression','SVR','Decision_Tree','RF','GBDT','stacking_regressor','voting_regressor'])
result_dataset

,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
median_absolute_error,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_squared_error,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
def metrics(y_true,y_pred,model = 'Knearest_neighbours'):
  r2_scores = r2_score(y_true,y_pred)
  p = 13 # number of features in the test dataset
  adjusted_r2_scores = 1- (1- r2_score(y_true,y_pred))*(len(y_true)-1)/(len(y_true)-p-1)
  median_absolute_errors = median_absolute_error(y_true,y_pred)
  mean_squared_errors = mean_squared_error(y_true,y_pred)
  mean_absolute_percentage_errors = mean_absolute_percentage_error(y_true,y_pred)
  mean_absolute_errors = mean_absolute_error(y_true,y_pred)
  result_dataset[model]['r2_score'] = r2_scores
  result_dataset[model]['adjusted_r2_score'] = adjusted_r2_scores
  result_dataset[model]['median_absolute_error'] = median_absolute_errors
  result_dataset[model]['mean_squared_error'] = mean_squared_errors
  result_dataset[model]['mean_absolute_percentage_error'] = mean_absolute_percentage_errors
  result_dataset[model]['mean_absolute_error'] = mean_absolute_errors
  print('r2_score for '+ model + " :" + str(r2_scores))
  print('adjusted_r2_score for '+ model + " :" + str(adjusted_r2_scores)) # Adj r2 = 1-(1-R2)*(n-1)/(n-p-1) Where n is the sample size and p is the number of independent variables.
  print('median_absolute_error for '+ model + " :" + str(median_absolute_errors))
  print('mean_squared_error for '+ model + " :" + str(mean_squared_errors))
  print('mean_absolute_percentage_error for '+ model + " :" + str(mean_absolute_percentage_errors))
  print('mean_absolute_error for '+ model + " :" + str(mean_absolute_errors))

Now lets start with KNN Regressor

In [ ]:
knn_reg = KNeighborsRegressor()
param_grids = {'base_estimator__n_neighbors' : [1, 2, 3, 4, 5],'max_features' : [0.3,0.4,0.5,0.6,0.7],'n_estimators':[25,28,30,32,35],'max_samples' : [0.4,0.5,0.6,0.7,0.8]}
knn_reg = GridSearchCV(estimator = BaggingRegressor(knn_reg),param_grid = param_grids,cv = 5, scoring = 'r2',verbose = 2)
knn_reg.fit(X_train,y_train)
knn_reg.best_estimator_

BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=1),
                 max_features=0.7, max_samples=0.8, n_estimators=32)

In [11]:
knn_reg = KNeighborsRegressor(n_neighbors = 1)
knn_reg =BaggingRegressor(knn_reg,max_features=0.7,max_samples=0.8, n_estimators=32,n_jobs =-1)
knn_reg.fit(X_train,y_train)

BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=1),
                 max_features=0.7, max_samples=0.8, n_estimators=32, n_jobs=-1)

In [12]:
y_pred = knn_reg.predict(X_test)
metrics(y_test,y_pred,model = 'Knearest_neighbours')
result_dataset

r2_score for Knearest_neighbours :0.8157839988821631
adjusted_r2_score for Knearest_neighbours :0.7885702714443008
median_absolute_error for Knearest_neighbours :1.5062500000000032
mean_squared_error for Knearest_neighbours :13.509256280637254
mean_absolute_percentage_error for Knearest_neighbours :0.11612122617533943
mean_absolute_error for Knearest_neighbours :2.1995710784313727


,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.815784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.788570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
median_absolute_error,1.506250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_squared_error,13.509256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.116121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,2.199571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# lets implement linear regression

In [ ]:
linear_reg = LinearRegression()
param_grids = {'max_features' : [0.3,0.4,0.5,0.6,0.7],'n_estimators':[25,28,30,32,35],'max_samples' : [0.4,0.5,0.6,0.7,0.8]}
linear_reg = GridSearchCV(estimator = BaggingRegressor(linear_reg),param_grid = param_grids,cv = 5, scoring = 'r2',verbose = 2)
linear_reg.fit(X_train,y_train)
linear_reg.best_estimator_

BaggingRegressor(base_estimator=LinearRegression(), max_features=0.7,
                 max_samples=0.4, n_estimators=28)

In [13]:
linear_reg = LinearRegression()
linear_reg =BaggingRegressor(linear_reg,max_features=0.7,max_samples=0.4, n_estimators=28,n_jobs =-1)
linear_reg.fit(X_train,y_train)

BaggingRegressor(base_estimator=LinearRegression(), max_features=0.7,
                 max_samples=0.4, n_estimators=28, n_jobs=-1)

In [14]:
y_pred = linear_reg.predict(X_test)
metrics(y_test,y_pred,model = 'linear_regression')
result_dataset

r2_score for linear_regression :0.6563536554697026
adjusted_r2_score for linear_regression :0.605587718209545
median_absolute_error for linear_regression :1.888309786367584
mean_squared_error for linear_regression :25.200886513622468
mean_absolute_percentage_error for linear_regression :0.16287411621933448
mean_absolute_error for linear_regression :3.1293288261382526


,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.815784,0.656354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.788570,0.605588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
median_absolute_error,1.506250,1.888310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_squared_error,13.509256,25.200887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.116121,0.162874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,2.199571,3.129329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# polynomial linear regression



In [15]:
polynomial = PolynomialFeatures()
X_train_poly = polynomial.fit_transform(X_train)
X_test_poly =  polynomial.transform(X_test) 

In [16]:
print(X_train.shape)
print(X_train_poly.shape)
print(X_test.shape)
print(X_test_poly.shape)

(404, 13)
(404, 105)
(102, 13)
(102, 105)


In [ ]:
linear_reg = LinearRegression()
param_grids = {'max_features' : [0.3,0.4,0.5,0.6,0.7],'n_estimators':[25,28,30,32,35],'max_samples' : [0.4,0.5,0.6,0.7,0.8]}
linear_reg = GridSearchCV(estimator = BaggingRegressor(linear_reg),param_grid = param_grids,cv = 5, scoring = 'r2',verbose = 2)
linear_reg.fit(X_train_poly,y_train)
linear_reg.best_estimator_

BaggingRegressor(base_estimator=LinearRegression(), max_features=0.5,
                 max_samples=0.8, n_estimators=35)

In [19]:
linear_reg = LinearRegression()
linear_reg =BaggingRegressor(linear_reg,max_features=0.5,max_samples=0.8, n_estimators=35,n_jobs =-1)
linear_reg.fit(X_train_poly,y_train)

BaggingRegressor(base_estimator=LinearRegression(), max_features=0.5,
                 max_samples=0.8, n_estimators=35, n_jobs=-1)

In [20]:
y_pred = linear_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'polynomial_linear_regression')
result_dataset

r2_score for polynomial_linear_regression :0.8237693267063718
adjusted_r2_score for polynomial_linear_regression :0.7977352499698132
median_absolute_error for polynomial_linear_regression :1.4055806250312415
mean_squared_error for polynomial_linear_regression :12.923661981512648
mean_absolute_percentage_error for polynomial_linear_regression :0.11224923402574259
mean_absolute_error for polynomial_linear_regression :2.1374021288427243


,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.815784,0.656354,0.823769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.788570,0.605588,0.797735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
median_absolute_error,1.506250,1.888310,1.405581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_squared_error,13.509256,25.200887,12.923662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.116121,0.162874,0.112249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,2.199571,3.129329,2.137402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# lets start with Lasso regression

In [ ]:
lasso_reg = Lasso(random_state = 42)
param_grids = {'base_estimator__alpha' : [0.0001, 0.001, 0.00001,0.1,0.01],'max_features' : [0.3,0.4,0.5,0.6,0.7],'n_estimators':[25,28,30,32,35],'max_samples' : [0.4,0.5,0.6,0.7,0.8]}
lasso_reg = GridSearchCV(estimator = BaggingRegressor(lasso_reg),param_grid = param_grids,cv = 5, scoring = 'r2',verbose = 2)
lasso_reg.fit(X_train_poly,y_train)
lasso_reg.best_estimator_

BaggingRegressor(base_estimator=Lasso(alpha=0.01, random_state=42),
                 max_features=0.7, max_samples=0.8, n_estimators=28)

In [32]:
lasso_reg =  Lasso(random_state = 42,alpha = 0.01)
lasso_reg =BaggingRegressor(lasso_reg,max_features=0.7,max_samples=0.8, n_estimators=26,n_jobs =-1)
lasso_reg.fit(X_train_poly,y_train)

BaggingRegressor(base_estimator=Lasso(alpha=0.01, random_state=42),
                 max_features=0.7, max_samples=0.8, n_estimators=26, n_jobs=-1)

In [33]:
y_pred = lasso_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'lasso_regression')
result_dataset

r2_score for lasso_regression :0.8333146897067225
adjusted_r2_score for lasso_regression :0.8086907234133974
median_absolute_error for lasso_regression :1.4513300819184192
mean_squared_error for lasso_regression :12.223664401058354
mean_absolute_percentage_error for lasso_regression :0.11328451176277719
mean_absolute_error for lasso_regression :2.1688857371424426


,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.815784,0.656354,0.823769,0.833315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.788570,0.605588,0.797735,0.808691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
median_absolute_error,1.506250,1.888310,1.405581,1.451330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_squared_error,13.509256,25.200887,12.923662,12.223664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.116121,0.162874,0.112249,0.113285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,2.199571,3.129329,2.137402,2.168886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# lets start with Ridge regression

In [ ]:
ridge_reg = Ridge(random_state = 42)
param_grids = {'base_estimator__alpha' : [1.0,0.001,0.1,0.01],'max_features' : [0.3,0.4,0.5,0.6,0.7],'n_estimators':[25,28,30,32,35],'max_samples' : [0.4,0.5,0.6,0.7,0.8]}
ridge_reg = GridSearchCV(estimator = BaggingRegressor(ridge_reg),param_grid = param_grids,cv = 5, scoring = 'r2',verbose = 2)
ridge_reg.fit(X_train_poly,y_train)
ridge_reg.best_estimator_

BaggingRegressor(base_estimator=Ridge(alpha=0.1, random_state=42),
                 max_features=0.6, max_samples=0.8, n_estimators=28)

In [34]:
ridge_reg =  Ridge(random_state = 42,alpha = 0.1)
ridge_reg =BaggingRegressor(ridge_reg,max_features=0.6,max_samples=0.8, n_estimators=28,n_jobs =-1)
ridge_reg.fit(X_train_poly,y_train)

BaggingRegressor(base_estimator=Ridge(alpha=0.1, random_state=42),
                 max_features=0.6, max_samples=0.8, n_estimators=28, n_jobs=-1)

In [35]:
y_pred = ridge_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'ridge_regression')
result_dataset

r2_score for ridge_regression :0.8417007992553824
adjusted_r2_score for ridge_regression :0.818315690054473
median_absolute_error for ridge_regression :1.4676537246681551
mean_squared_error for ridge_regression :11.608679261858223
mean_absolute_percentage_error for ridge_regression :0.12156724370112197
mean_absolute_error for ridge_regression :2.1782203500575976


,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.815784,0.656354,0.823769,0.833315,0.841701,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.788570,0.605588,0.797735,0.808691,0.818316,0.0,0.0,0.0,0.0,0.0,0.0,0.0
median_absolute_error,1.506250,1.888310,1.405581,1.451330,1.467654,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_squared_error,13.509256,25.200887,12.923662,12.223664,11.608679,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.116121,0.162874,0.112249,0.113285,0.121567,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,2.199571,3.129329,2.137402,2.168886,2.178220,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# lets implement Elastic Net regression

In [ ]:
elastic_reg = ElasticNet(random_state = 42)
param_grids = {'base_estimator__alpha' : [1.0,0.001,0.1,0.01],'base_estimator__l1_ratio' : [0.1,0.4,0.7,0.9],'max_features' : [0.3,0.4,0.5,0.6,0.7],'n_estimators':[25,28,30,32,35],'max_samples' : [0.4,0.5,0.6,0.7,0.8]}
elastic_reg = GridSearchCV(estimator = BaggingRegressor(elastic_reg),param_grid = param_grids,cv = 5, scoring = 'r2',verbose = 2)
elastic_reg.fit(X_train_poly,y_train)
elastic_reg.best_estimator_

BaggingRegressor(base_estimator=ElasticNet(alpha=0.001, l1_ratio=0.7,
                                           random_state=42),
                 max_features=0.7, max_samples=0.8, n_estimators=28)

In [36]:
elastic_reg =  ElasticNet(random_state = 42,alpha = 0.001,l1_ratio=0.7)
elastic_reg =BaggingRegressor(elastic_reg,max_features=0.7,max_samples=0.8, n_estimators=28,n_jobs =-1)
elastic_reg.fit(X_train_poly,y_train)

BaggingRegressor(base_estimator=ElasticNet(alpha=0.001, l1_ratio=0.7,
                                           random_state=42),
                 max_features=0.7, max_samples=0.8, n_estimators=28, n_jobs=-1)

In [37]:
y_pred = elastic_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'elasticnet_regression')
result_dataset

r2_score for elasticnet_regression :0.8464366274206894
adjusted_r2_score for elasticnet_regression :0.8237511291987458
median_absolute_error for elasticnet_regression :1.5488679604117586
mean_squared_error for elasticnet_regression :11.261383066099054
mean_absolute_percentage_error for elasticnet_regression :0.10162588035211255
mean_absolute_error for elasticnet_regression :2.0246390015260856


,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.815784,0.656354,0.823769,0.833315,0.841701,0.846437,0.0,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.788570,0.605588,0.797735,0.808691,0.818316,0.823751,0.0,0.0,0.0,0.0,0.0,0.0
median_absolute_error,1.506250,1.888310,1.405581,1.451330,1.467654,1.548868,0.0,0.0,0.0,0.0,0.0,0.0
mean_squared_error,13.509256,25.200887,12.923662,12.223664,11.608679,11.261383,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.116121,0.162874,0.112249,0.113285,0.121567,0.101626,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,2.199571,3.129329,2.137402,2.168886,2.178220,2.024639,0.0,0.0,0.0,0.0,0.0,0.0


# lets implement Support Vector regression

In [ ]:
svr_reg = SVR()
param_grids = {'base_estimator__C' : [1.0,0.001,0.1,0.01],'base_estimator__kernel' : ['linear','poly','rbf','sigmoid'],'max_features' : [0.4,0.5,0.6],'n_estimators':[28,30,32],'max_samples' : [0.5,0.6,0.7]}
svr_reg = GridSearchCV(estimator = BaggingRegressor(svr_reg),param_grid = param_grids,cv = 5, scoring = 'r2',verbose = 2)
svr_reg.fit(X_train,y_train)
svr_reg.best_estimator_

BaggingRegressor(base_estimator=SVR(kernel='linear'), max_features=0.6,
                 max_samples=0.6, n_estimators=30)

In [38]:
svr_reg =  SVR(kernel='linear')
svr_reg =BaggingRegressor(svr_reg,max_features=0.6,max_samples=0.6, n_estimators=30,n_jobs =-1)
svr_reg.fit(X_train_poly,y_train)

BaggingRegressor(base_estimator=SVR(kernel='linear'), max_features=0.6,
                 max_samples=0.6, n_estimators=30, n_jobs=-1)

In [39]:
y_pred = svr_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'SVR')
result_dataset

r2_score for SVR :0.816935322731844
adjusted_r2_score for SVR :0.7898916772263209
median_absolute_error for SVR :1.3688699205413997
mean_squared_error for SVR :13.424825347097453
mean_absolute_percentage_error for SVR :0.11238277740954172
mean_absolute_error for SVR :2.0724800720874175


,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.815784,0.656354,0.823769,0.833315,0.841701,0.846437,0.816935,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.788570,0.605588,0.797735,0.808691,0.818316,0.823751,0.789892,0.0,0.0,0.0,0.0,0.0
median_absolute_error,1.506250,1.888310,1.405581,1.451330,1.467654,1.548868,1.368870,0.0,0.0,0.0,0.0,0.0
mean_squared_error,13.509256,25.200887,12.923662,12.223664,11.608679,11.261383,13.424825,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.116121,0.162874,0.112249,0.113285,0.121567,0.101626,0.112383,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,2.199571,3.129329,2.137402,2.168886,2.178220,2.024639,2.072480,0.0,0.0,0.0,0.0,0.0


# lets implement Decision Tree Regression

In [ ]:
dt_reg = DecisionTreeRegressor()
param_grids = {'base_estimator__max_depth' : [17,20,22],'max_features' : [0.6,0.7,0.8],'n_estimators':[20,25,27],'max_samples' : [0.3,0.4,0.5]}
dt_reg = GridSearchCV(estimator = BaggingRegressor(dt_reg),param_grid = param_grids,cv = 5, scoring = 'r2',verbose = 2)
dt_reg.fit(X_train_poly,y_train)
dt_reg.best_estimator_

BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=22),
                 max_features=0.7, max_samples=0.5, n_estimators=25)

In [40]:
dt_reg =  DecisionTreeRegressor(max_depth=22)
dt_reg =BaggingRegressor(dt_reg,max_features=0.7,max_samples=0.5, n_estimators=25,n_jobs =-1)
dt_reg.fit(X_train_poly,y_train)

BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=22),
                 max_features=0.7, max_samples=0.5, n_estimators=25, n_jobs=-1)

In [41]:
y_pred = dt_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'Decision_Tree')
result_dataset

r2_score for Decision_Tree :0.8596914061080783
adjusted_r2_score for Decision_Tree :0.8389640001922262
median_absolute_error for Decision_Tree :1.452
mean_squared_error for Decision_Tree :10.289359999999999
mean_absolute_percentage_error for Decision_Tree :0.10946160774195168
mean_absolute_error for Decision_Tree :2.11521568627451


,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.815784,0.656354,0.823769,0.833315,0.841701,0.846437,0.816935,0.859691,0.0,0.0,0.0,0.0
adjusted_r2_score,0.788570,0.605588,0.797735,0.808691,0.818316,0.823751,0.789892,0.838964,0.0,0.0,0.0,0.0
median_absolute_error,1.506250,1.888310,1.405581,1.451330,1.467654,1.548868,1.368870,1.452000,0.0,0.0,0.0,0.0
mean_squared_error,13.509256,25.200887,12.923662,12.223664,11.608679,11.261383,13.424825,10.289360,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.116121,0.162874,0.112249,0.113285,0.121567,0.101626,0.112383,0.109462,0.0,0.0,0.0,0.0
mean_absolute_error,2.199571,3.129329,2.137402,2.168886,2.178220,2.024639,2.072480,2.115216,0.0,0.0,0.0,0.0


Now lets implement Randomforest Regressor

In [ ]:
rf_reg = RandomForestRegressor()
param_grids = {'n_estimators' : [25,50,75,100,125,150],'criterion' : ['squared_error', 'absolute_error'],'max_depth' : [45,50,60,70]}
rf_reg = GridSearchCV(estimator = rf_reg,param_grid = param_grids,cv = 5, scoring = 'r2', verbose = 2)
rf_reg.fit(X_train_poly,y_train)
rf_reg.best_estimator_

RandomForestRegressor(max_depth=60, n_estimators=75)

In [48]:
rf_reg = RandomForestRegressor(max_depth=60, n_estimators=75)
rf_reg.fit(X_train_poly,y_train)

RandomForestRegressor(max_depth=60, n_estimators=75)

In [49]:
y_pred = rf_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'RF')
result_dataset

r2_score for RF :0.8513066770290512
adjusted_r2_score for RF :0.8293406179537973
median_absolute_error for RF :1.542666666666662
mean_squared_error for RF :10.904243904139433
mean_absolute_percentage_error for RF :0.10894444598528742
mean_absolute_error for RF :2.1142745098039217


,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.815784,0.656354,0.823769,0.833315,0.841701,0.846437,0.816935,0.859691,0.851307,0.0,0.0,0.0
adjusted_r2_score,0.788570,0.605588,0.797735,0.808691,0.818316,0.823751,0.789892,0.838964,0.829341,0.0,0.0,0.0
median_absolute_error,1.506250,1.888310,1.405581,1.451330,1.467654,1.548868,1.368870,1.452000,1.542667,0.0,0.0,0.0
mean_squared_error,13.509256,25.200887,12.923662,12.223664,11.608679,11.261383,13.424825,10.289360,10.904244,0.0,0.0,0.0
mean_absolute_percentage_error,0.116121,0.162874,0.112249,0.113285,0.121567,0.101626,0.112383,0.109462,0.108944,0.0,0.0,0.0
mean_absolute_error,2.199571,3.129329,2.137402,2.168886,2.178220,2.024639,2.072480,2.115216,2.114275,0.0,0.0,0.0


# lets implement GBDT

In [ ]:
gbdt_reg = GradientBoostingRegressor(random_state = 42)
param_grids = {'loss' : ['squared_error', 'absolute_error', 'huber', 'quantile'],'learning_rate' : [0.1,0.3,0.6,0.9],'n_estimators':[80,90,100,110],'criterion':['friedman_mse','squared_error','mse'],'max_depth': [1,2,3,4,5]}
gbdt_reg = GridSearchCV(estimator = gbdt_reg,param_grid = param_grids,cv = 5, scoring = 'r2', verbose = 2)
gbdt_reg.fit(X_train,y_train)
gbdt_reg.best_estimator_

GradientBoostingRegressor(learning_rate=0.3, loss='huber', n_estimators=90,
                          random_state=42)

In [71]:
gbdt_reg = GradientBoostingRegressor(learning_rate=0.2, loss='squared_error', n_estimators=100, random_state=42,max_depth=4,criterion = 'mse')
gbdt_reg.fit(X_train_poly,y_train)
y_pred = gbdt_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'GBDT')
result_dataset

r2_score for GBDT :0.9101403180923426
adjusted_r2_score for GBDT :0.8968655923559842
median_absolute_error for GBDT :1.4735138941890167
mean_squared_error for GBDT :6.589750427871743
mean_absolute_percentage_error for GBDT :0.09516769383755921
mean_absolute_error for GBDT :1.8417779984194205


,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.815784,0.656354,0.823769,0.833315,0.841701,0.846437,0.816935,0.859691,0.851307,0.910140,0.0,0.0
adjusted_r2_score,0.788570,0.605588,0.797735,0.808691,0.818316,0.823751,0.789892,0.838964,0.829341,0.896866,0.0,0.0
median_absolute_error,1.506250,1.888310,1.405581,1.451330,1.467654,1.548868,1.368870,1.452000,1.542667,1.473514,0.0,0.0
mean_squared_error,13.509256,25.200887,12.923662,12.223664,11.608679,11.261383,13.424825,10.289360,10.904244,6.589750,0.0,0.0
mean_absolute_percentage_error,0.116121,0.162874,0.112249,0.113285,0.121567,0.101626,0.112383,0.109462,0.108944,0.095168,0.0,0.0
mean_absolute_error,2.199571,3.129329,2.137402,2.168886,2.178220,2.024639,2.072480,2.115216,2.114275,1.841778,0.0,0.0


# lets apply Stacking

In [74]:
estimators = [('knn',knn_reg),('linear',linear_reg),('lasso',lasso_reg),('ridge_reg',ridge_reg),('svr',svr_reg),('decisiontree',dt_reg),('randomforest',rf_reg),('gbdt',gbdt_reg)]
stacking_reg= StackingRegressor(estimators=estimators, final_estimator=gbdt_reg)
param_grids = {'passthrough':[True,False]}
stacking_reg = GridSearchCV(estimator = stacking_reg,param_grid = param_grids,cv = 5, scoring = 'r2', verbose = 2)
stacking_reg.fit(X_train_poly,y_train)
stacking_reg.best_estimator_




Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] END ...................................passthrough=True; total time=  21.4s
[CV] END ...................................passthrough=True; total time=  26.0s
[CV] END ...................................passthrough=True; total time=  18.0s
[CV] END ...................................passthrough=True; total time=  18.2s
[CV] END ...................................passthrough=True; total time=  18.7s
[CV] END ..................................passthrough=False; total time=  17.2s
[CV] END ..................................passthrough=False; total time=  17.0s
[CV] END ..................................passthrough=False; total time=  17.4s
[CV] END ..................................passthrough=False; total time=  17.8s
[CV] END ..................................passthrough=False; total time=  18.4s


StackingRegressor(estimators=[('knn',
                               BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=1),
                                                max_features=0.7,
                                                max_samples=0.8,
                                                n_estimators=32, n_jobs=-1)),
                              ('linear',
                               BaggingRegressor(base_estimator=LinearRegression(),
                                                max_features=0.5,
                                                max_samples=0.8,
                                                n_estimators=35, n_jobs=-1)),
                              ('lasso',
                               BaggingRegressor(base_estimator=Lasso(alpha=0.01,
                                                                     rando...
                                                max_features=0.7,
                                                max_samp

In [75]:
stacking_reg.best_params_

{'passthrough': True}

In [80]:
estimators = [('knn',knn_reg),('lasso',lasso_reg),('ridge_reg',ridge_reg),('svr',svr_reg),('decisiontree',dt_reg),('randomforest',rf_reg),('gbdt',gbdt_reg)]
stacking_reg= StackingRegressor(estimators=estimators, final_estimator=gbdt_reg,passthrough = True)
stacking_reg.fit(X_train_poly,y_train)

StackingRegressor(estimators=[('knn',
                               BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=1),
                                                max_features=0.7,
                                                max_samples=0.8,
                                                n_estimators=32, n_jobs=-1)),
                              ('lasso',
                               BaggingRegressor(base_estimator=Lasso(alpha=0.01,
                                                                     random_state=42),
                                                max_features=0.7,
                                                max_samples=0.8,
                                                n_estimators=26, n_jobs=-1)),
                              ('ridge_reg',
                               BaggingRegressor(base_estimator=Ridg...
                                                max_features=0.7,
                                                max_samp

In [81]:
y_pred = stacking_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'stacking_regressor')
result_dataset

r2_score for stacking_regressor :0.8327213837103561
adjusted_r2_score for stacking_regressor :0.808009769940295
median_absolute_error for stacking_regressor :1.3168138741869688
mean_squared_error for stacking_regressor :12.267173774343613
mean_absolute_percentage_error for stacking_regressor :0.10244492703654375
mean_absolute_error for stacking_regressor :2.10606657241062


,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.815784,0.656354,0.823769,0.833315,0.841701,0.846437,0.816935,0.859691,0.851307,0.910140,0.832721,0.0
adjusted_r2_score,0.788570,0.605588,0.797735,0.808691,0.818316,0.823751,0.789892,0.838964,0.829341,0.896866,0.808010,0.0
median_absolute_error,1.506250,1.888310,1.405581,1.451330,1.467654,1.548868,1.368870,1.452000,1.542667,1.473514,1.316814,0.0
mean_squared_error,13.509256,25.200887,12.923662,12.223664,11.608679,11.261383,13.424825,10.289360,10.904244,6.589750,12.267174,0.0
mean_absolute_percentage_error,0.116121,0.162874,0.112249,0.113285,0.121567,0.101626,0.112383,0.109462,0.108944,0.095168,0.102445,0.0
mean_absolute_error,2.199571,3.129329,2.137402,2.168886,2.178220,2.024639,2.072480,2.115216,2.114275,1.841778,2.106067,0.0


# now lets apply voting Regressor

In [83]:
estimators = [('knn',knn_reg),('lasso',lasso_reg),('ridge_reg',ridge_reg),('svr',svr_reg),('decisiontree',dt_reg),('randomforest',rf_reg),('gbdt',gbdt_reg)]
voting_reg = VotingRegressor(estimators=estimators)
voting_reg.fit(X_train_poly,y_train)

VotingRegressor(estimators=[('knn',
                             BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=1),
                                              max_features=0.7, max_samples=0.8,
                                              n_estimators=32, n_jobs=-1)),
                            ('lasso',
                             BaggingRegressor(base_estimator=Lasso(alpha=0.01,
                                                                   random_state=42),
                                              max_features=0.7, max_samples=0.8,
                                              n_estimators=26, n_jobs=-1)),
                            ('ridge_reg',
                             BaggingRegressor(base_estimator=Ridge(...
                                              max_features=0.6, max_samples=0.6,
                                              n_estimators=30, n_jobs=-1)),
                            ('decisiontree',
                             Bagging

In [84]:
y_pred = voting_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'voting_regressor')
result_dataset

r2_score for voting_regressor :0.8493696092179356
adjusted_r2_score for voting_regressor :0.8271173923978579
median_absolute_error for voting_regressor :1.4160391653483622
mean_squared_error for voting_regressor :11.046296414966621
mean_absolute_percentage_error for voting_regressor :0.09976177646567677
mean_absolute_error for voting_regressor :1.9573073027734986


,Knearest_neighbours,linear_regression,polynomial_linear_regression,lasso_regression,ridge_regression,elasticnet_regression,SVR,Decision_Tree,RF,GBDT,stacking_regressor,voting_regressor
r2_score,0.815784,0.656354,0.823769,0.833315,0.841701,0.846437,0.816935,0.859691,0.851307,0.910140,0.832721,0.849370
adjusted_r2_score,0.788570,0.605588,0.797735,0.808691,0.818316,0.823751,0.789892,0.838964,0.829341,0.896866,0.808010,0.827117
median_absolute_error,1.506250,1.888310,1.405581,1.451330,1.467654,1.548868,1.368870,1.452000,1.542667,1.473514,1.316814,1.416039
mean_squared_error,13.509256,25.200887,12.923662,12.223664,11.608679,11.261383,13.424825,10.289360,10.904244,6.589750,12.267174,11.046296
mean_absolute_percentage_error,0.116121,0.162874,0.112249,0.113285,0.121567,0.101626,0.112383,0.109462,0.108944,0.095168,0.102445,0.099762
mean_absolute_error,2.199571,3.129329,2.137402,2.168886,2.178220,2.024639,2.072480,2.115216,2.114275,1.841778,2.106067,1.957307
